In [1]:
import re
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from paddleocr import PaddleOCR
from skimage.feature import match_template

In [3]:
def non_max_suppression(boxes, scores, threshold):
    # Mengurutkan kotak dan skor berdasarkan skor secara menurun
    sorted_indices = np.argsort(scores)[::-1]
    boxes = boxes[sorted_indices]
    scores = scores[sorted_indices]

    # Menginisialisasi daftar kotak yang dipilih
    selected_boxes = []

    while len(boxes) > 0:
        # Mengambil kotak dengan skor tertinggi
        current_box = boxes[0]
        selected_boxes.append(current_box)

        # Menghitung IoU (Intersection over Union) antara kotak saat ini dan kotak yang tersisa
        ious = calculate_iou(current_box, boxes[1:])

        # Mengambil kotak yang memiliki IoU kurang dari threshold
        selected_indices = np.where(ious < threshold)[0]

        # Menghapus kotak yang dipilih dari daftar kotak
        boxes = boxes[selected_indices + 1]
        scores = scores[selected_indices + 1]

    return selected_boxes

def calculate_iou(box, boxes):
    # Menghitung luas kotak saat ini
    x1 = box[0]
    y1 = box[1]
    x2 = box[2]
    y2 = box[3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Menghitung luas kotak-kotak yang tersisa
    x1s = boxes[:, 0]
    y1s = boxes[:, 1]
    x2s = boxes[:, 2]
    y2s = boxes[:, 3]
    areas = (x2s - x1s + 1) * (y2s - y1s + 1)

    # Menghitung koordinat persekutuan antara kotak saat ini dan kotak-kotak yang tersisa
    xx1s = np.maximum(x1, x1s)
    yy1s = np.maximum(y1, y1s)
    xx2s = np.minimum(x2, x2s)
    yy2s = np.minimum(y2, y2s)

    # Menghitung luas persekutuan
    intersection = np.maximum(0, xx2s - xx1s + 1) * \
        np.maximum(0, yy2s - yy1s + 1)

    # Menghitung IoU (Intersection over Union)
    ious = intersection / (area + areas - intersection)

    return ious

def crop_image(image, start_x, start_y, end_x, end_y):
    # Memotong citra berdasarkan koordinat yang diberikan
    cropped_image = image[start_y:end_y, start_x:end_x]
    return cropped_image

def is_date(txt):
    pattern = r"\b\d{4}-\d{2}-\d{2}\b"
    tanggal = re.findall(pattern, txt)
    if tanggal:
        return True
    return False

def is_time(txt):
    pattern = r"\b\d{2}:\d{2}:\d{2}\b"
    waktu = re.findall(pattern, txt)
    if waktu:
        return True
    return False

def is_number(txt):
    pattern = r"\b(?:\d{1,3}(?:\.\d{3})*|\d+)(?:,\d{1,2})?\b"
    angka = re.findall(pattern, txt)
    if angka:
        return True
    return False

In [4]:
threshold = 0.2
threshold_nms = 0.5
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)

In [5]:
[line[1][0] for line in array_ls[0]]

NameError: name 'array_ls' is not defined

In [6]:
image_rgb = np.array(Image.open("./image/image.jpeg").convert("RGB"))
template_rgb = np.array(Image.open("./image/template.jpeg").convert("RGB"))
image_gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
template_gray = cv2.cvtColor(template_rgb, cv2.COLOR_RGB2GRAY)
result = match_template(image_gray, template_gray)
loc = np.where(result >= threshold)
boxes = np.column_stack((loc[1], loc[0], loc[1] + template_gray.shape[1], loc[0] + template_gray.shape[0]))
selected_boxes = non_max_suppression(boxes, result[loc], threshold_nms)
if len(selected_boxes) == 0:
    print("BUDI")
else:
    x1, y1, x2, y2 = selected_boxes[0]
    image_rgb = image_rgb[y1:, x1:]
    array = ocr.ocr(image_rgb, cls=True)[0]
    array = [line[1][0] for line in array]
    data = []
    start = 0
    end = 0
    for i, result in enumerate(array):
        if is_date(result):
            end = i+1
            data.append(
                array[start:end]
            )
            start = i+1
        arr_desc=[]
        arr_type=[]
        arr_date=[]
        arr_time=[]
        arr_cash=[]
        arr_saldo_akhir=[]
        for i, data_i in enumerate(data):
            array_time = []
            array_date = []
            array_word = []
            array_code = []
            array_nominal = []
            for data_j in data_i:
                if len(data_j) == 1 and "K" in data_j:
                    array_code.append(data_j)
                elif len(data_j) == 1 and "D" in data_j:
                    array_code.append(data_j)
                elif "." in data_j or "," in data_j or ",00" in data_j or ".00" in data_j:
                    array_nominal.append(data_j)
                elif is_time(data_j):
                    array_time.append(data_j)
                elif is_date(data_j):
                    array_date.append(data_j)
                else:
                    array_word.append(data_j)
        word = ' '.join(array_word)
        arr_desc.append(word)
        arr_type.append(array_code)
        arr_date.append(array_date)
        arr_time.append(array_time)
        arr_cash.append(array_nominal[0])
        arr_saldo_akhir.append(array_nominal[1])
table = {
    'Keterangan': arr_desc,
    'Tipe': arr_type,
    'Date': arr_date,
    'Time': arr_time,
    'Nominal': arr_cash,
    'Saldo Akhir': arr_saldo_akhir
}
table = pd.DataFrame(table)

FileNotFoundError: [Errno 2] No such file or directory: './image/template.jpeg'

In [14]:
array_ls[]

IndexError: list index out of range

In [10]:
from paddleocr import PaddleOCR,draw_ocr
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
img_path = 'image\hasil POI (Point Of Interest).jpg'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)


# draw result
from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
im_show = draw_ocr(image, boxes, txts, scores, font_path='./fonts/simfang.ttf')
im_show = Image.fromarray(im_show)
im_show.save('result.jpg')

[[[212.0, 1.0], [288.0, 4.0], [287.0, 23.0], [212.0, 21.0]], ('Riwayat', 0.995233952999115)]
[[[452.0, 3.0], [477.0, 3.0], [477.0, 31.0], [452.0, 31.0]], ('A', 0.9906746745109558)]
[[[10.0, 78.0], [58.0, 81.0], [57.0, 97.0], [9.0, 94.0]], ('Uraian', 0.9962136149406433)]
[[[209.0, 82.0], [239.0, 82.0], [239.0, 96.0], [209.0, 96.0]], ('Tipe', 0.9970061779022217)]
[[[284.0, 82.0], [341.0, 82.0], [341.0, 95.0], [284.0, 95.0]], ('Nominal', 0.9971173405647278)]
[[[397.0, 77.0], [475.0, 80.0], [475.0, 96.0], [396.0, 93.0]], ('Saldo Akhir', 0.9637417197227478)]
[[[4.0, 115.0], [156.0, 118.0], [156.0, 133.0], [3.0, 130.0]], ('TRF/PAY/TOP-UP ECHANNEL', 0.9694861769676208)]
[[[203.0, 117.0], [218.0, 117.0], [218.0, 132.0], [203.0, 132.0]], ('D', 0.9661812782287598)]
[[[282.0, 115.0], [342.0, 118.0], [341.0, 134.0], [281.0, 131.0]], ('650.065,00', 0.9734980463981628)]
[[[411.0, 115.0], [463.0, 118.0], [462.0, 134.0], [410.0, 131.0]], ('22,410,00', 0.9411954879760742)]
[[[6.0, 132.0], [163.0, 132.0

OSError: cannot open resource

In [1]:
a = 100

if a == 100 :
    b = 100
    
print(b)

100
